In [1]:
import h5py, os, torch
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader, Dataset
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

## Outline
 - Generate a specific model for each scene
     - Each location: 1600 images w/o presence, 800 imags w/ presence
 - Create Dataloader
     - 80-20 split for training/testing (validation?)
     - Each img (input size): 1024 x 720 px
 - Model Architecture
     - Modified VGG16?
     - Custom CNN architecture?
     - Concatenate pixel values and put in logistic regression?

---

## Model Base/Helpers/Train & Test functions

In [2]:
def accuracy(outs, labels):
    res = {}
    # check accuracy for 3 different thresholds
    for th in [.50, .75, .80]:
        outs_th = outs >= th
        # append onto result dictionary to be returned to function that called `accuracy()`
        res[th] = torch.tensor(torch.sum(outs_th == labels).item() / len(outs))
    return res

In [3]:
class ModelBase(nn.Module):
#     training step
    def train_step(self, batch):
        xb, labels = batch
#         xb = xb.float()
#         labels = labels.float()
        outs = self(xb)
        loss = F.binary_cross_entropy(outs, labels)
        return loss
#     validation step
    def val_step(self, batch):
        xb, labels = batch
#         xb = xb.float()
#         labels = labels.float()
        outs = self(xb)
        loss = F.binary_cross_entropy(outs, labels)
        acc = accuracy(outs, labels)
        return {'loss': loss.detach(), 'acc': acc}
#     validation epoch (avg accuracies and losses)
    def val_epoch_end(self, outputs):
        for i in range(10):
            print(outputs[i])
        batch_loss = [x['loss'] for x in outputs]
        avg_loss = torch.stack(batch_loss).mean()
        batch_acc50 = [x['acc'][.50] for x in outputs]
        batch_acc75 = [x['acc'][.75] for x in outputs]
        batch_acc80 = [x['acc'][.80] for x in outputs]
        avg_acc50 = torch.stack(batch_acc50).mean()
        avg_acc75 = torch.stack(batch_acc75).mean()
        avg_acc80 = torch.stack(batch_acc80).mean()
        return {'avg_loss': avg_loss, 'avg_acc': [avg_acc50, avg_acc75, avg_acc80]}
#     print everything important
    def epoch_end(self, epoch, avgs, test=False):
        s = 'test' if test else 'val'
        print(f'EPOCH {epoch + 1:<10} | {s}_loss:{avgs["avg_loss"]:.3f}, {s}_acc (threshold): (.50){avgs["avg_acc"][0]:.3f}, (.75){avgs["avg_acc"][1]:.3f}, (.80){avgs["avg_acc"][2]:.3f}')

In [4]:
@torch.no_grad()
def evaluate(model, val_dl):
    # eval mode
    model.eval()
    outputs = [model.val_step(batch) for batch in val_dl]
    return model.val_epoch_end(outputs)


def fit(epochs, lr, model, train_dl, val_dl, opt_func=torch.optim.Adam):
    torch.cuda.empty_cache()
    history = []
    # define optimizer
    optimizer = opt_func(model.parameters(), lr)
    # for each epoch...
    for epoch in range(epochs):
        # training mode
        model.train()
        # (training) for each batch in train_dl...
        for batch in tqdm(train_dl):
            # pass thru model
            loss = model.train_step(batch)
            # perform gradient descent
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # validation
        res = evaluate(model, val_dl)
        # print everything useful
        model.epoch_end(epoch, res, test=False)
        # append to history
        history.append(res)
    return history

---

## Models

In [5]:
class VGG16_PT(ModelBase):
    def __init__(self):
        super().__init__()
#         pretrained VGG16 model w/ batch norm
        self.network = torchvision.models.vgg16_bn(pretrained=True)
#         change first layer to accept only 1 dimension of color (b/w)
        self.network.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
#         change last fc layer to output one value
        self.network.classifier[6] = nn.Linear(4096, 1, bias=True)
        
    def forward(self, xb):
        out = self.network(xb)
        out = F.sigmoid(out)
        return out

In [6]:
class VGG11_PT(ModelBase):
    def __init__(self):
        super().__init__()
#         pretrained VGG16 model w/ batch norm
        self.network = torchvision.models.vgg11_bn(pretrained=True)
#         change first layer to accept only 1 dimension of color (b/w)
        self.network.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
#         change last fc layer to output one value
        self.network.classifier[6] = nn.Linear(4096, 1, bias=True)
        
    def forward(self, xb):
        out = self.network(xb)
        out = F.sigmoid(out)
        return out

In [7]:
# class Custom_NPT(ModelBase):
#     def __init__(self):
#         super().__init__()                                       # 1 x 1024 x 720
# #         custom-defined model w/o pretrained weights
#         self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)   # 8 x 1024 x 720
#         self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)  # 16 x 1024 x 720
#         self.pool1 = nn.MaxPool2d(2)                             # 16 x 512 x 360
#         self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # 32 x 512 x 360
#         self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1) # 32 x 512 x 360
#         self.pool2 = nn.MaxPool2d(2)                             # 32 x 256 x 180
#         self.conv5 = nn.Conv2d(32, 16, kernel_size=3, padding=1) # 16 x 256 x 180
#         self.pool3 = nn.MaxPool2d(2)                             # 8 x 256 x 180
#         self.fc1 = nn.Linear(8*256*180, 4096)
#         self.fc2 = nn.Linear(4096, 512)
#         self.fc3 = nn.Linear(512, 1)
        
#     def forward(self, xb):
#         out = F.relu(self.conv1(xb))
#         out = F.relu(self.conv2(out))
#         out = self.pool1(out)
#         out = F.relu(self.conv3(out))
#         out = F.relu(self.conv4(out))
#         out = self.pool2(out)
#         out = F.relu(self.conv5(out))
#         out = self.pool3(out)
        
#         out = torch.flatten(out, 1)
#         out = F.relu(self.fc1(out))

In [8]:
class VGG16_NPT(ModelBase):
    def __init__(self):
        super().__init__()
#         pretrained VGG16 model w/ batch norm
        self.network = torchvision.models.vgg16_bn(pretrained=False)
#         change first layer to accept only 1 dimension of color (b/w)
        self.network.features[0] = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
#         change last fc layer to output one value
        self.network.classifier[6] = nn.Linear(4096, 1, bias=True)
        
    def forward(self, xb):
        out = self.network(xb)
        out = F.sigmoid(out)
        return out

---

## DataSet & DataLoader

In [9]:
class Prudhoe_DS(Dataset):
    def __init__(self, h5_pth, transforms=None):
        self.transforms = transforms
        self.h5_pth = h5_pth
        self.mask_locs = []
        
        with h5py.File(self.h5_pth, 'r') as F:
            pos_shape = F['POS'].attrs['shape'][0]
            neg_shape = F['NEG'].attrs['shape'][0]
            for shape, key in zip([pos_shape, neg_shape], ['POS', 'NEG']):
                for i in range(shape):
                    self.mask_locs.append({'key': key,
                                          'key_idx': i})
        
    def __len__(self):
        return len(self.mask_locs)
    
    def __getitem__(self, idx):
        mask_loc = self.mask_locs[idx]
        with h5py.File(self.h5_pth, 'r') as F:
            mask = F[mask_loc['key']][mask_loc['key_idx']]
            mask = np.array(mask)
            
            if self.transforms:
                mask = self.transforms()(mask)
#             mask = mask.unsqueeze(0)
            
        label = 1 if mask_loc['key'] == 'POS' else 0
        return mask, label

In [10]:
# PTH = '/scratch/richardso21/20-21_BGSUB/'
# site = 'prudhoe_12'

In [11]:
# h5F = os.path.join(PTH, 'FgSegNet_O', f'{site}.h5')

In [12]:
# f = h5py.File(h5F, 'r')

In [13]:
# test = np.vstack([f['POS'],f['NEG']])

In [14]:
# test.shape

In [15]:
# y = np.expand_dims(test, axis=1)

In [16]:
# y.shape

In [17]:
# del y, test

In [18]:
# f.close()

In [19]:
class Prudhoe_DS_v2(Dataset):
    def __init__(self, h5_pth, transforms=None):
        self.transforms = transforms
        self.h5_pth = h5_pth
        self.masks = []
        self.labels = []
        
        with h5py.File(self.h5_pth, 'r') as F:
            pos = F['POS']
            neg = F['NEG']
            pos_len = pos.attrs['shape'][0]
            neg_len = neg.attrs['shape'][0]
            
            self.masks = np.expand_dims(np.vstack([pos, neg]), axis=1).astype('float32')
            self.labels = np.concatenate([np.ones(pos_len),
                                          np.zeros(neg_len)]).astype('float32')
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        mask = self.masks[idx]
        label = self.labels[idx]
        
        return mask, label

---

In [20]:
torch.manual_seed(42)

In [21]:
PTH = '/scratch/richardso21/20-21_BGSUB/'
site = 'prudhoe_12'

In [22]:
h5F = os.path.join(PTH, 'FgSegNet_O', f'{site}.h5')

In [23]:
ds = Prudhoe_DS_v2(h5F)
ds_size = len(ds)

In [24]:
train_size = int(ds_size * .70)
val_size = (ds_size - train_size) // 2
test_size = ds_size - train_size - val_size

In [25]:
train_ds, val_ds, test_ds = random_split(ds, [train_size, val_size, test_size])

In [26]:
batch_size = 4

In [27]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True, pin_memory=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size*2, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_ds, batch_size*2, pin_memory=True, num_workers=4)

In [28]:
# from torchvision.utils import make_grid

In [29]:
# for images, _ in train_loader:
#     print('images.shape:', images.shape)
#     print(_)
#     plt.figure(figsize=(24,12))
#     plt.axis('off')
#     plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
#     break

---

In [30]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

In [31]:
device = get_default_device()
device

device(type='cuda')

In [32]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [33]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

In [34]:
model = to_device(VGG11_PT(), device)

In [35]:
# model = model.double()

In [36]:
history = [evaluate(model, val_loader)]
history

/scratch/richardso21/conda/envs/fgsegnet/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/scratch/richardso21/conda/envs/fgsegnet/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])) is deprecated. Please ensure they have the same size.
  app.launch_new_instance()


{'loss': tensor(0.6872, device='cuda:0'), 'acc': {0.5: tensor(4.), 0.75: tensor(4.), 0.8: tensor(4.)}}
{'loss': tensor(0.6899, device='cuda:0'), 'acc': {0.5: tensor(6.), 0.75: tensor(6.), 0.8: tensor(6.)}}
{'loss': tensor(0.6913, device='cuda:0'), 'acc': {0.5: tensor(5.), 0.75: tensor(5.), 0.8: tensor(5.)}}
{'loss': tensor(0.6957, device='cuda:0'), 'acc': {0.5: tensor(4.), 0.75: tensor(4.), 0.8: tensor(4.)}}
{'loss': tensor(0.6875, device='cuda:0'), 'acc': {0.5: tensor(8.), 0.75: tensor(8.), 0.8: tensor(8.)}}
{'loss': tensor(0.6891, device='cuda:0'), 'acc': {0.5: tensor(6.2500), 0.75: tensor(7.), 0.8: tensor(7.)}}
{'loss': tensor(0.6909, device='cuda:0'), 'acc': {0.5: tensor(5.), 0.75: tensor(5.), 0.8: tensor(5.)}}
{'loss': tensor(0.6894, device='cuda:0'), 'acc': {0.5: tensor(6.), 0.75: tensor(6.), 0.8: tensor(6.)}}
{'loss': tensor(0.6900, device='cuda:0'), 'acc': {0.5: tensor(6.), 0.75: tensor(6.), 0.8: tensor(6.)}}
{'loss': tensor(0.6907, device='cuda:0'), 'acc': {0.5: tensor(6.), 0.

[{'avg_loss': tensor(0.6916, device='cuda:0'),
  'avg_acc': [tensor(5.1321), tensor(5.2642), tensor(5.2642)]}]

**TODO**: figure out accuracy bug!

In [37]:
# torch.cuda.empty_cache()